In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
# Custom imports:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_chain import InterviewChain

/Users/jamesaphoenix/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Topic Research:

In [9]:
%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

Note: you may need to restart the kernel to use updated packages.


In [10]:
!playwright install

In [23]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
# Constant variables:
TOPIC = "Neural networks"
os.environ["SERPAPI_API_KEY"] = ""
os.environ["STABILITY_API_KEY"] = ""

In [4]:
# Extract content from webpages into LangChain documents:
text_documents = await collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

In [5]:
# LLM, text splitter + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [6]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

Summarizing the data!
Summarizing the data!
Summarizing the data!
Time taken: 3.486579179763794
Finished summarizing the data!
---
Time taken: 3.627920150756836
Finished summarizing the data!
---
Time taken: 37.57901906967163
Finished summarizing the data!
---


---

## Expert Interview Questions:


In [7]:
interview_chain = InterviewChain(topic=TOPIC, document_summaries=summaries)
interview_questions = interview_chain()

----------------------------------------

## Answer The Interview Questions:

In [8]:
for question in interview_questions.questions:
    print(f"Answer the following question: {question.question}\n", flush=True)
    answer = input(f"Answer the following question: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Answer the following question: Can you explain the concept of neural networks and how they are related to machine learning?

------------------------------------------
Answer the following question: What is the role of training data in improving the accuracy of neural networks?

------------------------------------------
Answer the following question: How do neural networks make complex decisions based on previous outputs?

------------------------------------------
Answer the following question: What is the significance of sigmoid neurons and supervised learning in training neural networks?

------------------------------------------
Answer the following question: Can you provide examples of real-world applications where neural networks are used?

------------------------------------------


---

## General Article Outline:


In [9]:
from article_outline_generation import BlogOutlineGenerator

blog_outline_generator = BlogOutlineGenerator(topic=TOPIC, questions_and_answers=[item.dict()  
                                                                                  for item in interview_questions.questions ] )
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

Generating the outline...
---
Finished generating the outline!
---


---

## Article Text Generation:


In [11]:
from article_generation import ContentGenerator

content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)

# Vectorize and store the original webpages:
content_gen.split_and_vectorize_documents(text_documents)

# Create the blog post (this may take several minutes to run, so please be patient):
blog_post = content_gen.generate_blog_post()

Generating the blog post...
---
Finished generating the blog post!
---


In [12]:
blog_post

['# Neural Networks: An Overview\n\n## What are Neural Networks?\n\nNeural networks, also known as artificial neural networks (ANNs) or simulated neural networks (SNNs), are a subset of machine learning and are at the heart of deep learning algorithms. They are computational models inspired by the structure and functioning of the human brain. Neural networks consist of interconnected nodes, or "neurons," organized into layers.\n\nA neural network typically has three types of layers:\n1. Input Layer: This layer receives the initial data or input features. Each neuron in the input layer represents a feature of the data.\n2. Hidden Layers: These intermediate layers process the input data through a series of weighted connections and apply activation functions to introduce non-linearity. Hidden layers are responsible for learning complex patterns and representations from the data.\n3. Output Layer: The final layer produces the network\'s prediction or output. The number of neurons in the ou

-----

## Image Creation:

In [13]:
from image_generation_chain import create_image
image = create_image(outline_result.title)

In [16]:
print(f"The image is ready! The filepath is {image[0]}")

The image is ready! The filepath is c31daed.png
